In [1]:
filename = "log.nve.lammps"

In [148]:
from copy import deepcopy
class StorageObjectLAMMPS:
    """
    Object to restore the LAMMPS defaults

    Details
    -------
    active_dumps :
        empty dict
    active_fixes :
        empty dict
    integrator : 
        None
    thermostat :
        None
    barostat :
        None
    nsteps :
        None
    LAMMPS-Defaults
    ---------------
    run_style='verlet'
    """
    active_fixes=dict()  #
    active_dumps=dict()  #
    
    integrator=None
    barostat=None
    thermostat=None

    nsteps = None 
    
    run_style='verlet'
        
        

class LogFileReader(StorageObjectLAMMPS):
    def __init__(self,filename):
        """
        
        Attributes
        ----------
        runs :
            List of the run objects
        """
        self.filename = filename
       
        # some defaults
        self.runs=[]
        # parse logfile
        self.parse_file(self.filename) 
        
    def parse_file(self,filename):
        """
        Parse a lammps logfile
        """
        with open(filename) as fp:
            for line in fp:
                line_strip=line.strip()
                if len(line_strip) == 0: continue  # empty line
                if line_strip[0] == "#": continue  # comment line
                self.interprete_line(line)
        self.compile_infos()
        
    def interprete_line(self,line):
        """
        Function to interprete the line
        """
        line_split = line.split()
        keyword = line_split[0]
        # Handle fixes
        if   keyword in ['fix', 'unfix']:
            self.handle_fix(line_split)
        elif keyword == 'run':
            self.handle_run(line_split)
    
    def handle_fix(self,line_split):
        """
        Handle a fix & unfix
        """
        keyword  = line_split[0]
        if   keyword == 'fix':
            fix_id   = line_split[1]  # id
            fix_grp  = line_split[2]  # group
            fix_type = line_split[3]  # type
            fix_args = line_split[4:] # args 
            self.active_fixes[fix_id]=[fix_grp,fix_type,fix_args]
        elif keyword == 'unfix':
            del self.active_fixes[line_split[1]]
        
    def handle_run(self,line_split):
        """
        Add a run
        """
        
        self.compile_infos()
        
        run = StorageObjectLAMMPS()
        for key in vars(StorageObjectLAMMPS).keys():
            if not key.startswith("__"):
                setattr(run,key,getattr(self,key))
        run.nsteps = int(line_split[1])
        if self.nsteps is None:
            self.nsteps = 0
        self.nsteps+= int(line_split[1])
        self.runs.append(run)
    
    def compile_infos(self):
        """
        Function to compile the infos.
        Finds integrator, thermostat, barostat
        """
        self.integrator = None
        self.thermostat = None
        self.barostat = None
        # go over all fix IDS
        for fixid,(fix_grp, fix_type, fix_args) in self.active_fixes.iteritems():
            
            # check if have set an integrator
            if fix_type in ['nve', 'nvt', 'npt', 'nph']:
                assert self.integrator is None, "Multiple Integreators are not implemented"
                self.integrator = fix_type
            
            # check for thermostat
            for part in fix_type.split("/"):  # get rid of combinations
                if part in ['nvt', 'npt', 'temp', 'langevin']:
                    assert self.thermostat is None, "Multiple Thermostats are not implemented"
                    self.thermostat = fix_type
            
            # check for barostat
            for part in fix_type.split("/"):  # get rid of combinations
                if part in ['nph', 'npt', 'press']:
                    assert self.barostat is None, "Multiple Barostats are not implemented"
                    self.barostat = fix_type
            

In [149]:
L = LogFileReader("log.nve.lammps")
for run in L.runs:
    print run.__dict__

{'barostat': None, 'thermostat': None, 'run_style': 'verlet', 'integrator': 'nve', 'active_fixes': {'1': ['all', 'nve', []]}, 'active_dumps': {}, 'nsteps': 100}


In [150]:
L = LogFileReader("log.nvt.lammps")
for run in L.runs:
    print run.__dict__

{'barostat': None, 'thermostat': 'nvt', 'run_style': 'verlet', 'integrator': 'nvt', 'active_fixes': {'1': ['all', 'nvt', ['temp', '1', '1', '1']]}, 'active_dumps': {}, 'nsteps': 100}
